In [2]:
# API 키를 환경변수로 관리하기 위한 설정 파일
from dotenv import load_dotenv

# API 키 정보 로드
load_dotenv()

True

In [3]:
import os

In [4]:
import bs4
from langchain import hub
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import Chroma, FAISS
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_openai import ChatOpenAI, OpenAIEmbeddings

USER_AGENT environment variable not set, consider setting it to identify your requests.


In [5]:
from glob import glob

In [6]:
# !pip install chromadb

In [7]:
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate, ChatPromptTemplate, MessagesPlaceholder
from langchain_core.output_parsers import StrOutputParser

from langchain_core.runnables import RunnablePassthrough

from langchain.vectorstores import Chroma
from langchain_upstage import ChatUpstage, UpstageEmbeddings


In [39]:
patient_id = '남A'
persist_directory = f'.cache/db/{patient_id}'

In [40]:
llm = ChatUpstage(temperature=0, api_key=os.getenv("UPSTAGE_API_KEY"))


# Embeddings setup
embeddings = UpstageEmbeddings(
  api_key=os.getenv("UPSTAGE_API_KEY"),
  model="solar-embedding-1-large"
)

vectordb = Chroma(
    persist_directory=f'.cache/db/{patient_id}',
    embedding_function=embeddings
)

retriever = vectordb.as_retriever()

RAG_PROMPT_TEMPLATE="""

# Your role
    - You are a compassionate, articulate physician.

------
    
# Instructions
    - Your goal is to explain medical information in a way that is easy for your patients to understand, avoiding complex medical jargon as much as possible.
    - Given a medical document or chart, it's your job to explain the key information that the patient or their family asks about in a patient-friendly format. 
    - When specific details are provided, such as diagnosis codes or medical history, simplify these terms and explain them in a way that is easy to understand.

------

# Document: \n\t{context}

------

# Question: \n\t{question}

------

# IMPORTANT 
    - Answer in KOREAN
    - Let us know the metadata to the document you referenced

# Answer :
"""
# from TEMPLATES.rag_template import prompt
# rag_prompt = ChatPromptTemplate.from_template(prompt)
rag_prompt = ChatPromptTemplate.from_template(RAG_PROMPT_TEMPLATE)

def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

# Define the RAG chain
qa_chain = (
    {
        "context": retriever | format_docs,
        "question": RunnablePassthrough(),
    }
    | rag_prompt
    | llm
    | StrOutputParser()
)

In [41]:
query = "가장 최근 검사의 전반적인 결과가 어떻게 될까?"
print(qa_chain.invoke(query))

가장 최근 검사는 2024년 6월 24일 10시에 시행된 것으로 보입니다. 해당 검사에서 환자의 혈압(BP)은 107/51, 맥박(P)은 61, 호흡(RESP)은 20, 체온(T)은 36.5도로 측정되었습니다. 이 검사 결과는 환자의 상태를 파악하는 데 도움이 되지만, 전체적인 건강 상태를 판단하기 위해서는 다른 검사 결과와 함께 고려되어야 합니다.


In [26]:
print(qa_chain.invoke(query))

안녕하세요, 보호자님. 아버님께서 [질환들 나열]로 입원하셨을 때 얼마나 걱정이 크셨을까요? 정말 많은 어려움이 있으셨겠어요. 보호자님도 많이 힘드셨겠죠? 고생 많으셨습니다.

다행히 아버님께서 [현재 상태]을 보여주시고 있어서 조금 안심이 되실 것 같아요. [치료 방법 나열] 받고 계시니 아버님께서 빠르게 회복하시기를 진심으로 바라고 있어요.

의료진이 최선을 다해 치료하고 있으니 아버님이 하루하루 조금씩 나아지실 거라 믿습니다. 보호자님께서도 이 힘든 시기를 잘 이겨내시길 진심으로 바라며, 언제든지 도움이 필요하시거나 궁금한 점이 있으시면 말씀해 주세요. 함께 힘내요!

아버님의 가장 최근 검사에 대해서는, 의료진이 검사 결과를 분석하고 있을 것입니다. 검사 결과에 대한 자세한 내용은 의료진에게 문의하시는 것이 좋을 것 같아요. 보호자님께서도 아버님의 상태를 지속적으로 관찰하시고, 의료진의 지시에 따라 적절한 조치를 취해주시기 바랍니다.

보호자님도 많이 힘드실 텐데, 마음 깊이 위로의 말씀을 전하고 싶습니다. 필요하실 때 언제든지 말씀해 주세요. 함께 이겨냅시다!


In [27]:
# llm = ChatUpstage(api_key=os.getenv("UPSTAGE_API_KEY"))


# # Embeddings setup
# embeddings = UpstageEmbeddings(
#   api_key=os.getenv("UPSTAGE_API_KEY"),
#   model="solar-embedding-1-large"
# )

# vectordb = Chroma(
#     persist_directory=persist_directory,
#     embedding_function=embeddings
# )

# retriever = vectordb.as_retriever()

# RAG_PROMPT_TEMPLATE='''
# ### 시스템 설명
# 당신은 병원 AI 지원 시스템입니다. 보호자 또는 환자가 의료 상태와 관련된 질문을 하면, 환자의 최신 의료 기록을 기반으로 적절한 답변을 제공해야 합니다.
# 보호자와 환자에게 알기 쉽게 친절하게 답변해주세요.
# 아래에 환자의 의료 기록이 제공됩니다.

# ### 의료 기록
# {context}

# ### 질문
# {question}

# ### 응답
# 환자의 상태, 식사, 약물 복용, 거동 상태 등을 포함하여 질문에 대한 정확한 답변을 생성하십시오.
# 참고한 데이터의 메타데이터를 알려주세요
# '''

# rag_prompt = ChatPromptTemplate.from_template(RAG_PROMPT_TEMPLATE)

# def format_docs(docs):
#     return "\n\n".join(doc.page_content for doc in docs)

# # Define the RAG chain
# qa_chain = (
#     {
#         "context": retriever | format_docs,
#         "question": RunnablePassthrough(),
#     }
#     | rag_prompt
#     | llm
#     | StrOutputParser()
# )

In [28]:
query = "가장 최근 검사의 전반적인 결과가 어떻게 될까?"
print(qa_chain.invoke(query))

안녕하세요, 보호자님. 아버님께서 [질환들 나열]로 입원하셨을 때 얼마나 걱정이 크셨을까요? 정말 많은 어려움이 있으셨겠어요. 보호자님도 많이 힘드셨겠죠? 고생 많으셨습니다.

다행히 아버님께서 [현재 상태]을 보여주시고 있어서 조금 안심이 되실 것 같아요. [치료 방법 나열] 받고 계시니 아버님께서 빠르게 회복하시기를 진심으로 바라고 있어요.

의료진이 최선을 다해 치료하고 있으니 아버님이 하루하루 조금씩 나아지실 거라 믿습니다. 보호자님께서도 이 힘든 시기를 잘 이겨내시길 진심으로 바라며, 언제든지 도움이 필요하시거나 궁금한 점이 있으시면 말씀해 주세요. 함께 힘내요!

아버님의 가장 최근 검사에 대해서는, 의료진이 검사 결과를 분석하고 있을 것입니다. 검사 결과에 대한 자세한 내용은 의료진에게 문의하시는 것이 좋을 것 같아요. 보호자님께서도 아버님의 상태를 지속적으로 관찰하시고, 의료진의 지시에 따라 적절한 조치를 취해주시기 바랍니다.

보호자님도 많이 힘드실 텐데, 마음 깊이 위로의 말씀을 전하고 싶습니다. 필요하실 때 언제든지 말씀해 주세요. 함께 이겨냅시다!


In [29]:
print(qa_chain.invoke('말씀은 잘 하세요?'))

안녕하세요, 보호자님. 아버님께서 [뇌경색, 편마비]로 입원하셨을 때 얼마나 걱정이 크셨을까요? 정말 많은 어려움이 있으셨겠어요. 보호자님도 많이 힘드셨겠죠? 고생 많으셨습니다.

다행히 아버님께서는 현재 [와상상태, 왼쪽 편마비]로 L-tube (18Fr)를 유지하고 계시지만, 경관식을 통해 영양을 공급받고 계십니다. 대소변 조절은 아직 어려우셔서 기저귀를 착용하고 계십니다.

아버님께서 [말하기]에 어떤 어려움이 있으신지, 의료진에게 문의해보시는 것이 좋을 것 같습니다. 의료진은 아버님의 상태를 평가하고, 필요한 치료나 재활을 제안해 줄 수 있을 것입니다.

보호자님도 많이 힘드실 텐데, 언제든지 도움이 필요하시거나 궁금한 점이 있으시면 말씀해주세요. 함께 힘내요!

보호자님, 힘내세요. 아버님께서 조금씩 나아지실 거라 믿습니다. 언제든지 필요하신 것이나 도움이 필요하시면 말씀해주세요. 보호자님도 힘내세요, 제가 항상 응원하고 있을게요!


In [30]:
print(qa_chain.invoke('밤에 잘 주무세요?'))

안녕하세요, 보호자님. 아버님께서 [뇌경색, 폐렴]으로 입원하셨을 때 얼마나 걱정이 크셨을까요? 정말 많은 어려움이 있으셨겠어요. 보호자님도 많이 힘드셨겠죠? 고생 많으셨습니다.

다행히 아버님께서는 현재 [안정적인 상태]를 유지하고 계십니다. [항생제 치료, 산소 치료]를 받고 계시며, 의료진들은 아버님의 상태를 지속적으로 모니터링하고 있습니다.

아버님께서 밤에 주무시는 동안에는 [수면제]를 투여하여 편안한 수면을 도와드리고 있습니다. 의료진들은 아버님의 상태를 주의 깊게 관찰하고 있으며, 필요한 조치를 취하고 있습니다.

아버님께서 조금씩 회복되고 계시는 모습을 보면서 저도 기쁘고 안심이 됩니다. 언제든지 필요하신 것이 있으시면 말씀해주세요. 보호자님도 힘내세요, 제가 항상 응원하고 있을게요!

보호자님도 많이 힘드실 텐데, 마음 깊이 위로의 말씀을 전하고 싶습니다. 필요하실 때 언제든지 말씀해주세요. 함께 이겨내요!


In [31]:
print(qa_chain.invoke('거동은 어떠세요?'))

안녕하세요, 보호자님. 아버님께서 [왼쪽 편마비]로 인해 거동이 제한적이신 것으로 알고 있습니다. 의료진들은 아버님의 상태를 지속적으로 관찰하고 있으며, 아버님께서 최대한 편안하게 지내실 수 있도록 최선을 다하고 있습니다.

아버님께서는 [와상상태]로 침대에 누워 계시는 중이며, 스스로 움직이시기는 어려운 상황입니다. 그러나 의료진들은 아버님의 체위를 정기적으로 변경하여 욕창을 예방하고, 가능한 한 편안한 자세를 유지하도록 노력하고 있습니다. 또한, 아버님께서는 [L-tube (18Fr) kept]와 [경관식 장솔루션 400-400-400kcal/day given]을 통해 영양을 공급받고 계십니다.

아버님께서 거동이 제한적이지만, 의료진들은 아버님의 상태를 개선하기 위해 다양한 치료와 재활 프로그램을 제공하고 있습니다. 아버님께서 조금씩 회복하고 계시는 모습을 볼 수 있을 것입니다.

아버님의 상태를 고려하여 의료진들은 아버님을 돌보는 데 최선을 다하고 있습니다. 언제든지 궁금한 사항이나 도움이 필요하시면 언제든지 말씀해주세요. 함께 아버님의 건강을 위해 노력하겠습니다.


In [32]:
print(qa_chain.invoke('약은 잘 드세요?'))

안녕하세요, 보호자님. 아버님께서 입원하신 후로 많이 힘드셨죠? 정말로 많은 어려움이 있으셨을 것 같아요. 보호자님도 많이 힘드셨죠? 고생 많으셨습니다.

다행히도 아버님께서는 현재 약을 잘 드시고 계십니다. 아버님께서는 트라젠타정, 텔핀스타정, 네비스톨정, 영진아스피린장용정, 파모티딘정, 리보트릴정, 아세틸캡슐, 코푸시럽 등의 약물을 복용하고 계십니다. 이 약물들은 아버님의 상태를 관리하고 증상을 완화시키는 데 도움을 주고 있습니다.

아버님께서 약을 잘 드시고 계시다는 것은 정말 좋은 소식입니다. 의료진들은 아버님의 상태를 지속적으로 관찰하고 있으며, 필요한 경우 약물 조절을 진행하고 있습니다. 아버님께서 빠르게 회복하실 수 있도록 최선을 다하고 있습니다.

보호자님도 많이 힘드실 텐데, 언제든지 도움이 필요하시거나 궁금한 점이 있으시면 언제든지 말씀해주세요. 함께 아버님의 건강을 위해 노력하겠습니다. 힘내세요!

아버님께서 약을 잘 드시고 계시다는 것은 정말 좋은 소식입니다. 의료진들은 아버님의 상태를 지속적으로 관찰하고 있으며, 필요한 경우 약물 조절을 진행하고 있습니다. 아버님께서 빠르게 회복하실 수 있도록 최선을 다하고 있습니다.

보호자님도 많이 힘드실 텐데, 언제든지 도움이 필요하시거나 궁금한 점이 있으시면 언제든지 말씀해주세요. 함께 아버님의 건강을 위해 노력하겠습니다. 힘내세요!

아버님께서 약을 잘 드시고 계시다는 것은 정말 좋은 소식입니다. 의료진들은 아버님의 상태를 지속적으로 관찰하고 있으며, 필요한 경우 약물 조절을 진행하고 있습니다. 아버님께서 빠르게 회복하실 수 있도록 최선을 다하고 있습니다.

보호자님도 많이 힘드실 텐데, 언제든지 도움이 필요하시거나 궁금한 점이 있으시면 언제든지 말씀해주세요. 함께 아버님의 건강을 위해 노력하겠습니다. 힘내세요!

아버님께서 약을 잘 드시고 계시다는 것은 정말 좋은 소식입니다. 의료진들은 아버님의 상태를 지속적으로 관찰하고 있으며, 필요한 경우 약물 조절을 진행하고 있습니다. 아버님께서 빠르게 

In [33]:
print(qa_chain.invoke('많이 아파하지는 않으신 가요?'))

안녕하세요, 보호자님. 아버님께서 [뇌내출혈, 수면 개시 및 유지 장애, 상세불명의 위장염 및 결장염, 인후통(급성), 음식 또는 구토물에 의한 폐렴, 상세불명의 편마비, 폐부종, 기타 및 상세불명의 원발성 고혈압, 합병증을 동반하지 않은 2형 당뇨병, 상세불명의 고지질혈증, 욕창궤양 및 압박부위 제2단계, 반코마이신 내성, 카바페넴계, 기타 세균성 폐렴]으로 입원하셨을 때 얼마나 걱정이 크셨을까요? 정말 많은 어려움이 있으셨겠어요. 보호자님도 많이 힘드셨겠죠? 고생 많으셨습니다.

다행히 아버님께서는 현재 [와상상태 및 왼쪽 편마비]로 계시지만, 의료진들이 최선을 다해 치료하고 있습니다. 아버님께서 [코푸시럽]을 처방받고 계시며, [그린비아 장솔루션] 경관식을 통해 영양을 공급받고 계십니다. 또한, [욕창]에 대한 관리도 이루어지고 있습니다.

현재 아버님께서는 [인후통(급성)]과 [음식 또는 구토물에 의한 폐렴]으로 인해 약간의 불편함을 겪고 계실 수 있습니다. 하지만 의료진들이 적절한 치료를 제공하고 있으며, 아버님께서 잘 견뎌내고 계십니다.

아버님께서 조금이라도 편안하게 지내실 수 있도록 저희 의료진은 최선을 다하고 있습니다. 언제든지 보호자님의 의견이나 궁금한 점이 있으시면 언제든지 말씀해주세요. 함께 아버님의 회복을 위해 노력하겠습니다. 힘내세요!

보호자님께서도 많이 힘드실 텐데, 마음 깊이 위로의 말씀을 전하고 싶습니다. 필요하실 때 언제든지 말씀해주세요. 함께 이겨냅시다!


In [34]:
print(qa_chain.invoke('많이 아파하지는 않으신 가요?'))

안녕하세요, 보호자님. 아버님께서 [뇌내출혈, 수면 개시 및 유지 장애, 상세불명 기원의 위장염 및 결장염, 인후통(급성), 음식 또는 구토물에 의한 폐렴, 상세불명의 편마비, 폐부종, 기타 및 상세불명의 원발성 고혈압, 합병증을 동반하지 않은 2형 당뇨병, 상세불명의 고지질혈증, 욕창궤양 및 압박부위 제2단계, 반코마이신 내성, 카바페넴계, 기타 세균성 폐렴]으로 입원하셨을 때 얼마나 걱정이 크셨을까요? 정말 많은 어려움이 있으셨겠어요. 보호자님도 많이 힘드셨겠죠? 고생 많으셨습니다.

다행히 아버님께서는 현재 [와상상태 및 왼쪽 편마비]로 계시고, [L-tube (18Fr) kept & 경관식 400-400-400kcal/day, 대소변 조절 못해 기저귀 착용 중임 (기저귀마다 변이 조금씩 매달려 나옴), 계속적으로 오른손을 침대 난간을 툭툭치며 반복적인 행동 양상 보임, 깨어 있을때 항상 침상 두드리며 반복적인 행동장애 보임]으로 관찰되고 있습니다. 의료진이 최선을 다해 치료하고 있으니 아버님께서 하루하루 조금씩 나아지실 거라 믿습니다.

현재 아버님께서는 [통증 상태]로 많이 힘드셨겠어요. 현재 [통증에 대한 상세한 정보]를 가지고 있지만, 약물 치료로 잘 조절하고 있어요. 통증의 횟수도 줄어들었다니 정말 다행입니다.

이 힘든 시간에도 아버님께서 조금씩 회복하시는 모습에 저도 함께 기쁘고 안심이 됩니다. 언제든지 필요하시면 말씀해주세요. 보호자님도 힘내세요, 제가 항상 응원하고 있을게요!

보호자님께서도 많이 힘드실 텐데, 마음 깊이 위로의 말씀을 전하고 싶습니다. 필요하실 때 언제든지 말씀해주세요. 함께 이겨냅시다!


In [35]:
q = '아버님은 어떠세요?'
qa_chain.invoke(q)

'안녕하세요, 보호자님. 아버님께서 [뇌경색, 폐렴, 요로감염]으로 입원하셨을 때 얼마나 걱정이 크셨을까요? 정말 많은 어려움이 있으셨겠어요. 보호자님도 많이 힘드셨겠죠? 고생 많으셨습니다.\n\n다행히 아버님께서는 현재 [안정적인 상태]를 유지하고 계십니다. [항생제 치료, 산소 치료, 수액 치료]를 받고 계시며, 의료진들은 아버님의 회복을 위해 최선을 다하고 있습니다.\n\n아버님께서는 [오른쪽 편마비]와 [언어장애]가 있지만, 물리치료와 언어치료를 통해 조금씩 회복하고 계십니다. 또한, [식사]와 [대소변]은 [경관식]과 [도뇨관]을 통해 조절하고 있습니다.\n\n아버님께서는 [통증]은 [약물 치료]로 잘 조절되고 있습니다. [통증의 횟수]도 줄어들고 있어서 정말 다행입니다.\n\n아버님께서 [정서적인 안정]을 위해 보호자님의 따뜻한 관심과 사랑이 필요합니다. 언제든지 아버님의 상태에 대해 궁금한 점이 있으시면 언제든지 말씀해주세요. 함께 아버님의 회복을 위해 노력하겠습니다. 보호자님도 힘내세요!\n\n보호자님, 아버님의 빠른 회복을 진심으로 기원합니다. 언제든지 필요하신 것이 있으면 말씀해주세요. 함께 이겨낼 수 있을 거예요.'

In [42]:
text =  "안녕하세요, 보호자님. 아버님께서 [질환들 나열]로 입원하셨을 때 얼마나 걱정이 크셨을까요? 정말 많은 어려움이 있으셨겠어요. 보호자님도 많이 힘드셨겠죠? 고생 많으셨습니다.\n\n다행히 아버님께서 [현재 상태]을 보여주시고 있어서 조금 안심이 되실 것 같아요. [치료 방법 나열] 받고 계시니 아버님께서 빠르게 회복하시기를 진심으로 바라고 있어요. 의료진이 최선을 다해 치료하고 있으니 아버님이 하루하루 조금씩 나아지실 거라 믿습니다.\n\n보호자님께서도 이 힘든 시기를 잘 이겨내시길 진심으로 바라며, 언제든지 도움이 필요하시거나 궁금한 점이 있으시면 말씀해 주세요. 함께 힘내요!"

In [45]:
text.replace(r'[','').replace(r']','')

'안녕하세요, 보호자님. 아버님께서 질환들 나열로 입원하셨을 때 얼마나 걱정이 크셨을까요? 정말 많은 어려움이 있으셨겠어요. 보호자님도 많이 힘드셨겠죠? 고생 많으셨습니다.\n\n다행히 아버님께서 현재 상태을 보여주시고 있어서 조금 안심이 되실 것 같아요. 치료 방법 나열 받고 계시니 아버님께서 빠르게 회복하시기를 진심으로 바라고 있어요. 의료진이 최선을 다해 치료하고 있으니 아버님이 하루하루 조금씩 나아지실 거라 믿습니다.\n\n보호자님께서도 이 힘든 시기를 잘 이겨내시길 진심으로 바라며, 언제든지 도움이 필요하시거나 궁금한 점이 있으시면 말씀해 주세요. 함께 힘내요!'

In [46]:
text.replace(r'[|]','')

'안녕하세요, 보호자님. 아버님께서 [질환들 나열]로 입원하셨을 때 얼마나 걱정이 크셨을까요? 정말 많은 어려움이 있으셨겠어요. 보호자님도 많이 힘드셨겠죠? 고생 많으셨습니다.\n\n다행히 아버님께서 [현재 상태]을 보여주시고 있어서 조금 안심이 되실 것 같아요. [치료 방법 나열] 받고 계시니 아버님께서 빠르게 회복하시기를 진심으로 바라고 있어요. 의료진이 최선을 다해 치료하고 있으니 아버님이 하루하루 조금씩 나아지실 거라 믿습니다.\n\n보호자님께서도 이 힘든 시기를 잘 이겨내시길 진심으로 바라며, 언제든지 도움이 필요하시거나 궁금한 점이 있으시면 말씀해 주세요. 함께 힘내요!'